In [4]:
from kubernetes import client, config, stream
import re
import concurrent.futures
import time

# Load Kubernetes configuration
config.load_kube_config()

def measure_bandwidth_from_source_to_target(v1, namespace, source_pod, target_pod, test_duration=5):
    """Measure bandwidth from source pod to target pod."""
    source_name = source_pod.metadata.name
    target_ip = target_pod.status.pod_ip
    target_name = target_pod.metadata.name
    source_node = source_pod.spec.node_name
    target_node = target_pod.spec.node_name

    if source_name == target_name:
        return None  # Skip self-tests

    command = ['iperf3', '-c', target_ip, '-t', str(test_duration)]
    try:
        output = stream.stream(v1.connect_get_namespaced_pod_exec,
                               source_name,
                               namespace,
                               command=command,
                               stderr=True,
                               stdin=False,
                               stdout=True,
                               tty=False)
        # Parse the output for bandwidth
        match = re.search(r'(\d+\.?\d*\s[MKG]bits/sec)', output)
        bandwidth = match.group(1) if match else "Parsing Error"

        # Print the result immediately
        print(f"{source_node} -> {target_node}: {bandwidth}")
        return source_node, target_node, bandwidth

    except Exception as e:
        print(f"Error measuring bandwidth from {source_name} to {target_name}: {e}")
        return source_node, target_node, "Error"

def measure_bandwidth(namespace='measure-nodes-bd', max_concurrent_tasks=3, test_duration=5):
    """Measure bandwidth between all pods in the namespace."""
    v1 = client.CoreV1Api()
    pods = v1.list_namespaced_pod(namespace, label_selector="app=bandwidth-measurement").items

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_concurrent_tasks) as executor:
        futures = [
            executor.submit(measure_bandwidth_from_source_to_target, v1, namespace, src, tgt, test_duration)
            for src in pods for tgt in pods if src.metadata.name != tgt.metadata.name
        ]

        # Collect and print results as they complete
        results = {}
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                src_node, tgt_node, bandwidth = result
                if src_node not in results:
                    results[src_node] = {}
                results[src_node][tgt_node] = bandwidth

    return results

if __name__ == '__main__':
    namespace = 'measure-nodes-bd'
    print("Measuring bandwidth...")
    time.sleep(10)  # Allow some time for the DaemonSet to deploy
    results = measure_bandwidth(namespace=namespace, max_concurrent_tasks= 4, test_duration=10)
    print("\nFinal Cross-Node Bandwidth Results:")
    for src_node, targets in results.items():
        for tgt_node, bandwidth in targets.items():
            print(f"{src_node} -> {tgt_node}: {bandwidth}")


Measuring bandwidth...
k8s-worker-3 -> k8s-worker-8: 802 Mbits/sec
k8s-worker-3 -> k8s-worker-1: 717 Mbits/sec
k8s-worker-3 -> k8s-worker-5: 490 Mbits/sec
k8s-worker-3 -> k8s-worker-9: 348 Mbits/sec
k8s-worker-3 -> k8s-worker-6: 1.10 Gbits/sec
k8s-worker-3 -> k8s-worker-2: 446 Mbits/sec
k8s-worker-3 -> k8s-worker-7: 516 Mbits/sec
k8s-worker-3 -> k8s-worker-4: 360 Mbits/sec
k8s-worker-1 -> k8s-worker-3: 710 Mbits/sec
k8s-worker-1 -> k8s-worker-5: 455 Mbits/sec
k8s-worker-1 -> k8s-worker-8: 637 Mbits/sec
k8s-worker-1 -> k8s-worker-9: 612 Mbits/sec
k8s-worker-1 -> k8s-worker-6: 856 Mbits/sec
k8s-worker-1 -> k8s-worker-2: 554 Mbits/sec
k8s-worker-1 -> k8s-worker-7: 370 Mbits/sec
k8s-worker-1 -> k8s-worker-4: 800 Mbits/sec
k8s-worker-8 -> k8s-worker-3: 1.24 Gbits/sec
k8s-worker-8 -> k8s-worker-1: 890 Mbits/sec
k8s-worker-8 -> k8s-worker-5: 865 Mbits/sec
k8s-worker-8 -> k8s-worker-9: 1.00 Gbits/sec
k8s-worker-8 -> k8s-worker-6: 1.61 Gbits/sec
k8s-worker-8 -> k8s-worker-7: 977 Mbits/sec
k8s-w

In [5]:
import pandas as pd

# Convert the nested dictionary into a pandas DataFrame
df = pd.DataFrame(results)

# Transpose the DataFrame to align the source and destination workers as per convention
df = df.T

# Fill diagonal with 0's for self-latency (optional, if desired for clarity)
for worker in df.columns:
    df.at[worker, worker] = 0

# Save the DataFrame to a CSV file, and add the date, hour and minute to the file name
import datetime
now = datetime.datetime.now()
filename = f"bandwidth_results_{now.strftime('%Y-%m-%d_%H-%M')}.csv"
df.to_csv("/home/ubuntu/iDynamics/iBandwidth/measurer/data/"+filename)
df

,k8s-worker-8,k8s-worker-1,k8s-worker-5,k8s-worker-9,k8s-worker-6,k8s-worker-2,k8s-worker-7,k8s-worker-4,k8s-worker-3
k8s-worker-3,802 Mbits/sec,717 Mbits/sec,490 Mbits/sec,348 Mbits/sec,1.10 Gbits/sec,446 Mbits/sec,516 Mbits/sec,360 Mbits/sec,0
k8s-worker-1,637 Mbits/sec,0,455 Mbits/sec,612 Mbits/sec,856 Mbits/sec,554 Mbits/sec,370 Mbits/sec,800 Mbits/sec,710 Mbits/sec
k8s-worker-8,0,890 Mbits/sec,865 Mbits/sec,1.00 Gbits/sec,1.61 Gbits/sec,888 Mbits/sec,977 Mbits/sec,851 Mbits/sec,1.24 Gbits/sec
k8s-worker-5,1.20 Gbits/sec,1.15 Gbits/sec,0,1.51 Gbits/sec,2.25 Gbits/sec,1.14 Gbits/sec,1.11 Gbits/sec,1.10 Gbits/sec,1.93 Gbits/sec
k8s-worker-9,417 Mbits/sec,588 Mbits/sec,534 Mbits/sec,0,905 Mbits/sec,466 Mbits/sec,478 Mbits/sec,525 Mbits/sec,874 Mbits/sec
k8s-worker-6,1.33 Gbits/sec,900 Mbits/sec,954 Mbits/sec,1.93 Gbits/sec,0,946 Mbits/sec,1.02 Gbits/sec,1.12 Gbits/sec,1.73 Gbits/sec
k8s-worker-2,696 Mbits/sec,577 Mbits/sec,602 Mbits/sec,861 Mbits/sec,414 Mbits/sec,0,709 Mbits/sec,755 Mbits/sec,865 Mbits/sec
k8s-worker-7,575 Mbits/sec,476 Mbits/sec,772 Mbits/sec,813 Mbits/sec,513 Mbits/sec,717 Mbits/sec,0,673 Mbits/sec,1.02 Gbits/sec
k8s-worker-4,514 Mbits/sec,480 Mbits/sec,594 Mbits/sec,533 Mbits/sec,493 Mbits/sec,799 Mbits/sec,324 Mbits/sec,0,604 Mbits/sec
